# QGAR - A Flashcard Generating NLP Model

This notebook shows how to load and use the `QGAR` model.

Please read the [README](./readme.md) before continueing!

**Table of Contents:**
1. [Load QGAR](#load-qgar)
2. [Download and Preprocess SQuAD Dataset](#download-and-preprocess-squad-dataset)
3. [Run QGAR](#run-qgar)
4. [Train QGAR](#4-train-qgar)

</br>

---

</br>

# 0. Used Libraries

In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install transformers
%pip install datasets
%pip install wandb

## 1. Load QG
First, we must load the `QG` model and tokenizer.

In [1]:
from models.qg import QG

qg = QG("t5-small", "t5-small")
model = qg._model
tokenizer = qg._tokenizer

## 2. Download and Preprocess SQuAD Dataset

First, we download and preprocess the modified `SQuAD` dataset, adding separator (`<sep>`) and end of sequence tokens (`<\s>`) to each entry.
The preprocessed file is split in two datasets, `training` and `validation`, and the sets are saved in the `data` directory in `PyTorch` format under the names:
* [training_data.pt](./data/training_data.pt)
* [validation_data.pt](./data/validation_data.pt)

In [ ]:
%load_ext autoreload
%autoreload 2

from preprocessing.preprocessor import SquadPreprocessor

preprocessor = SquadPreprocessor(model, tokenizer)
preprocessor.preprocess_dataset()

## 3. Run QG
Next, we import `QG` and set up a pipeline.
Now, we can simply pass a context to the model to generate questions.

In [2]:
%load_ext autoreload
%autoreload 2

import json

questions = qg("Historical Fiction is one of those sub-genres of literature that takes many forms. It's most important feature, though, is that it's set in the past, with every element of the story conforming to the norms of the day. Here's how we define Historical Fiction, a look at its origins, and some popular types.")
print(json.dumps(questions, indent=4))

{
    "context": "Historical Fiction is one of those sub-genres of literature that takes many forms. It's most important feature, though, is that it's set in the past, with every element of the story conforming to the norms of the day. Here's how we define Historical Fiction, a look at its origins, and some popular types.",
    "questions": []
}


## 4. Train QG
To train `QG`, we first parse `settings.json` to get the training arguments.

We then call `train` which will train the model and push it to `The Coorporation`'s Huggingface Hub.

In [3]:
%load_ext autoreload
%autoreload 2

from main import parse_settings, get_wandb_token

%env WANDB_PROJECT=t5-qg

model_args, data_args, train_args = parse_settings()
qg.train(train_args, data_args, get_wandb_token())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: WANDB_PROJECT=t5-qg


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
Cloning https://huggingface.co/the-coorporation/t5-qg into local empty directory.


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

</br>

---